In [35]:
import sys 
import os
import subprocess
import numpy as np 

output_dir = '../output/run_clixo/'; 
if not os.path.exists(output_dir): os.makedirs(output_dir)
stringdb_dir = '../output/extracting_string_interactions/'

In [36]:
# Obtaining the clixo program 
#clixo_script='/mnt/c/Users/Anubhav/Desktop/mhk7-clixo_0.3-0362bea/mhk7-clixo_0.3-0362bea/clixo'
clixo_script = '/gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo'

# Listing the string interactions file 
interactions_fn = os.path.join(stringdb_dir, 'string_synapse_interactions.tsv')

In [ ]:
alpha = 0.1
beta = 0.1

# Naming the clixo output (can change if you need)
clixo_fn = os.path.join(output_dir, 
    'string_synapse_interactions_combined_score.clixo_alpha{}_beta{}.txt'.format(alpha, beta))
cmd = '{} {} {} {} > {}'.format(clixo_script, interactions_fn, alpha, beta, clixo_fn)
print('Running: {}'.format(cmd))

if not os.path.exists(clixo_fn): 
    sp = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=True)
    msgs = sp.communicate()
    print('\n'.join([x.decode() for x in msgs]))

Running: /gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo ../output/extracting_string_interactions/string_synapse_interactions.tsv 0.1 0.1 > ../output/run_clixo/string_synapse_interactions_combined_score.clixo_alpha0.1_beta0.1.txt


### Testing different values of alpha and beta

In [22]:
for alpha in np.arange(0, 0.9, 0.1): 
    for beta in np.arange(0, 0.9, 0.1): 
    
        cmd = '{} {} {} {} > {}'.format(clixo_script, interactions_fn, alpha, beta, clixo_fn)
        print('Running: {}'.format(cmd))

#         sp = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=True)
#         msgs = sp.communicate()
#         print('\n'.join([x.decode() for x in msgs]))
        break
    break

Running: /gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo ../output/stringdb/string_synapse_interactions_combined_score.tsv 0.0 0.0 > ../output/clixo/string_synapse_interactions_combined_score.clixo_alpha0.75_beta0.75.txt
